In [1]:
import numpy as np
from sklearn import preprocessing
import pandas as pd
import os
from tqdm import tqdm
import torch
from matplotlib import pyplot as plt
import time
import os
import torch.nn as nn
import torch.nn.utils.rnn as rnn
from torch.utils.data import Dataset, DataLoader, BatchSampler, RandomSampler
import librosa

%matplotlib inline

In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [ ]:
labels = pd.read_csv("trainingData.csv")
le = preprocessing.LabelEncoder()
le.fit(labels["Language"])
labels["Language"] = le.transform(labels["Language"])
labels.head()

In [ ]:
train_dirs = []
train_base_dir = "Training data"
for direc in os.listdir("Training data"):
  train_dirs.append(direc)

In [ ]:
train_labels = []
# train_data = []
pbar = tqdm(total=66176)
for i, direc in enumerate(train_dirs):
    label = labels[labels["Sample Filename"] == direc]["Language"].item()
#     y, sr = librosa.load(train_base_dir + "/" + direc)
#     train_data.append(librosa.feature.melspectrogram(y=y, sr=sr))
    train_labels.append(label)
    pbar.update(1)
pbar.close()    
labels = np.array(train_labels)
# train_data = np.array(train_data)
# np.save("train_data.npy", train_data)

In [ ]:
np.save("labels.npy", train_labels)

In [4]:
labels = np.load("labels.npy")

In [5]:
data = np.load("traindata.npy")

In [6]:
# data = data[:,:,50:250]
print(data.shape)
max(labels)

(66176, 40, 430)


175

In [7]:
test_size = 6000
train_size = data.shape[0] - test_size
print(train_size)
train = data[:train_size, :, :]
train_label = labels[:train_size]
test = data[train_size:, :, :]
test_label = labels[train_size:]

60176


In [8]:
print(train.shape)
# train_label = np.array(train_label)
print(train_label.shape)
train_label[0]

(60176, 40, 430)
(60176,)


104

In [9]:
print(test.shape)
# test_label = np.array(test_label)
test_label.shape
min(test_label)

(6000, 40, 430)


0

In [28]:
class TrainDataset(Dataset):
  
    def __init__(self, train, label):
        self.train = train
        self.label = label

    def __len__(self):
        return self.train.shape[0]

    def __getitem__(self, idx):
        train = torch.Tensor(self.train[idx])
        train = train.transpose(0,1)
        train = train.to(DEVICE)
        label = self.label[idx]
#         label = label + 1
        label = label.astype(np.float)
#         print(label)
        label = torch.tensor(label)
        label = label.to(DEVICE)
        return(train, label)

In [29]:
class TestDataset(Dataset):
  
  def __init__(self, test):
    self.test = test
  
  def __len__(self):
    return self.test.shape[0]
  
  def __getitem__(self, idx):
    test = torch.Tensor(self.test[idx])
    test = test.to(DEVICE)
#     test = test.unsqueeze(0)
    return test

In [30]:
train_dataset = TrainDataset(train, train_label)
test_dataset = TestDataset(test)
train_batch_size = 100
test_batch_size = 100
train_loader = DataLoader(train_dataset,
                        batch_sampler=BatchSampler(RandomSampler(train_dataset), train_batch_size, False))
test_loader = DataLoader(test_dataset, batch_size=test_batch_size)

In [31]:
# a = iter(train_loader).next()[1]
# a = list(a)
# torch.cat(a)
train_dataset[0][1]

tensor(104., device='cuda:0')

In [ ]:
class DetectionModel(nn.Module):
    
    def __init__(self,vocab_size,embed_size,hidden_size, nlayers):
        super(DetectionModel,self).__init__()
        self.vocab_size=vocab_size
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.nlayers=nlayers
        self.rnn = nn.LSTM(input_size = embed_size,hidden_size=hidden_size,num_layers=nlayers, bidirectional=True, batch_first=True) # Recurrent network
        self.scoring = nn.Linear(hidden_size * 2,vocab_size) # Projection layer
        
    def forward(self, seq_batch):
        batch_size = seq_batch.size(0)
#         print(batch_size)
        embed = seq_batch #L x N x E
        hidden = None
        output_lstm,hidden = self.rnn(embed,hidden) #L x N x H
#         print(output_lstm.shape)
#         output_lstm_flatten = output_lstm.contiguous().view(-1,self.hidden_size * 2) #(L*N) x H
#         output_flatten = self.scoring(output_lstm) #(L*N) x V
        return output_lstm

In [39]:
def train_epoch(model, optimizer, train_loader):
    criterion = nn.CrossEntropyLoss()
    criterion = criterion.to(DEVICE)
    batch_id=0
    for inputs,targets in train_loader:
        batch_loss = []
        batch_id+=1
#         inputs = inputs.to(DEVICE)
#         targets = targets.to(DEVICE)
        outputs = model(inputs) # 3D
#         print(outputs.shape)
#         outputs = outputs[:, -1, :] # pull out the last layer
        outputs = torch.sum(outputs, dim=1)
#         print(outputs.shape)
#         print(torch.argmax(outputs, dim=1))
#         print(targets)
#         print(targets.shape)
#         print(targets)
        loss = criterion(outputs,targets.long()) # Loss of the flattened outputs
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        batch_loss.append(loss.item())
#         print(loss.item())
        torch.cuda.empty_cache()
        if batch_id % 20 == 0:
            lpw = np.mean(batch_loss)
            torch.save(model, "model.pt")
            print("At batch",batch_id)
            print("Training loss :",lpw)

    return model
    

In [42]:
langcount = 176
model = DetectionModel(langcount,40,176,5)
# model = torch.load("model.pt")
model = model.to(DEVICE)
# optimizer = torch.optim.Adam(model.parameters(),lr=0.01, weight_decay=1e-6)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

/jet/var/python/lib/python3.6/site-packages/torch/backends/cudnn/__init__.py:89: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "


In [ ]:
for i in range(50):
    train_epoch(model, optimizer, train_loader)

/jet/var/python/lib/python3.6/site-packages/torch/backends/cudnn/__init__.py:89: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "
/jet/var/python/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type DetectionModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


At batch 20
Training loss : 18.6175479889
At batch 40
Training loss : 8.58160591125
At batch 60
Training loss : 6.46694469452
At batch 80
Training loss : 5.85277271271
At batch 100
Training loss : 5.95144033432
At batch 120
Training loss : 5.69115924835
At batch 140
Training loss : 5.72989702225


In [ ]:
torch.cuda.empty_cache()

In [ ]:
print(torch.backends.cudnn.version())

In [ ]:
print(torch.cuda.is_available())